# Remote tensors

In [2]:
import syft as sy
import torch as th


In [3]:
hook = sy.TorchHook(th)


In [5]:
viper = sy.VirtualWorker(hook, id = "viper")
quassi = sy.VirtualWorker(hook, id = "quassi")

In [6]:
x = th.tensor([1, 2, 3, 4, 5]).send(viper)
y = th.tensor([2, 2, 2, 2, 2]).send(quassi)

In [8]:
z = x + y #raises error because x and y are not on same machine

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:viper #objects:1> while the other is on <VirtualWorker id:quassi #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

In [9]:
x = th.tensor([1, 2, 3, 4, 5]).send(viper)
y = th.tensor([2, 2, 2, 2, 2]).send(viper)

In [12]:
z = x + y #now treats as the individaul tensor

In [13]:
z.get()

tensor([3, 4, 5, 6, 7])

In [14]:
z = th.add(x ,y) #same as x + y
z

(Wrapper)>[PointerTensor | me:36629187458 -> viper:32466870951]

In [15]:
z.get()

tensor([3, 4, 5, 6, 7])

# Simple linear model

In [26]:
inputs = th.tensor([[0., 1], [0, 0], [1, 1], [1, 0]], requires_grad = True).send(viper)
target = th.tensor([[1.], [0], [1], [0]], requires_grad = True).send(viper)

In [27]:
weights = th.tensor([[0.], [0]], requires_grad = True).send(viper)

In [29]:
for i in range(10):

    pred = inputs.mm(weights)

    loss = ((pred - target)**2).sum() #mse

    loss.backward() #backporpagate

    weights.data.sub_(weights.grad * 0.1) #wt update
    weights.grad *= 0

    print(loss.get().data)

tensor(0.0058)
tensor(0.0037)
tensor(0.0024)
tensor(0.0015)
tensor(0.0010)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)
